# Project on 20 Newsgroups’ dataset and use LDA to extract the naturally discussed topics.

### Download nltk stopwords and spacy model

In [6]:

import nltk; nltk.download('stopwords')

!pip install spacy download en

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\soulo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  Using cached download-0.3.5-py3-none-any.whl (8.8 kB)


ERROR: Could not find a version that satisfies the requirement en (from versions: none)
ERROR: No matching distribution found for en


### Import Packages

In [7]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

### Prepare Stopwords

In [9]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

### Import Newsgroups Data

In [11]:
# Import Dataset
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')

print(df.target_names.unique())

df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


###  Remove emails and newline characters

In [13]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


### Tokenize words and Clean-up text

In [15]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


### Creating Bigram and Trigram Models

Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring.

In [18]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  


# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


### Remove Stopwords, Make Bigrams and Lemmatize

In [20]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

### Let’s call the functions in order.

In [23]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# nlp = spacy.load('en', disable=['parser', 'ner'])
nlp = spacy.load('en_core_web_sm' , disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['s', 'thing', 'car', 'nntp_poste', 'host', 'rac_wam', 'university', 'park', 'line', 'wonder', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


### Create the Dictionary and Corpus needed for Topic Modeling

In [25]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 5), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)]]


In [26]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('body', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_poste', 1),
  ('park', 1),
  ('production', 1),
  ('rac_wam', 1),
  ('really', 1),
  ('rest', 1),
  ('s', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('spec', 1),
  ('sport', 1),
  ('thank', 1),
  ('thing', 1),
  ('university', 1),
  ('wonder', 1),
  ('year', 1)]]

### Building the Topic Model

In [28]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

### View the topics in LDA model

In [30]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.021*"research" + 0.019*"information" + 0.019*"high" + 0.019*"report" + '
  '0.018*"player" + 0.016*"service" + 0.015*"rate" + 0.014*"design" + '
  '0.013*"season" + 0.012*"low"'),
 (1,
  '0.077*"team" + 0.072*"game" + 0.053*"play" + 0.050*"faith" + 0.049*"win" + '
  '0.031*"belief" + 0.025*"atheist" + 0.025*"year" + 0.018*"wing" + '
  '0.018*"score"'),
 (2,
  '0.106*"space" + 0.029*"notice" + 0.029*"launch" + 0.026*"earth" + '
  '0.024*"mission" + 0.024*"orbit" + 0.023*"external" + 0.020*"vehicle" + '
  '0.019*"satellite" + 0.019*"door"'),
 (3,
  '0.022*"say" + 0.019*"people" + 0.017*"reason" + 0.017*"believe" + '
  '0.015*"evidence" + 0.014*"mean" + 0.012*"point" + 0.012*"question" + '
  '0.011*"many" + 0.010*"claim"'),
 (4,
  '0.078*"book" + 0.044*"science" + 0.042*"reference" + 0.036*"pin" + '
  '0.032*"section" + 0.025*"faq" + 0.024*"author" + 0.023*"copy" + '
  '0.023*"reality" + 0.022*"internal"'),
 (5,
  '0.065*"cost" + 0.059*"model" + 0.039*"character" + 0.036*"pictur

### Compute Model Perplexity and Coherence Score

In [32]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -13.32461339253533

Coherence Score:  0.483541481988623


There you have a coherence score of 0.53.

### Visualize the topics-keywords

In [35]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18     0.336524 -0.053716       1        1  34.413768
3      0.316929 -0.118905       2        1  15.585696
11     0.216786  0.285238       3        1   9.181682
6      0.247091  0.167005       4        1   9.110720
13     0.184108 -0.279645       5        1   7.614203
10    -0.019302  0.039638       6        1   5.391939
0      0.153916  0.027983       7        1   5.120650
9     -0.008545 -0.117702       8        1   2.140508
1      0.006923 -0.062096       9        1   2.065611
19     0.038372  0.069508      10        1   1.847952
16    -0.037562  0.037040      11        1   1.732312
4     -0.046036 -0.038089      12        1   1.606861
2     -0.081906  0.028035      13        1   1.237493
5     -0.118578  0.023845      14        1   1.144285
8     -0.162684  0.013118      15        1   0.619224
17    -0.175944 -0.007820      16        1   0.433673
15    -0.204259 -0.001777      17        1   0.294853
7     -0.212202 -0.003263      18        1   0.200450
14    -0.214082 -0.003631      19        1   0.158955
12    -0.219547 -0.004767      20        1   0.099166, topic_info=              Term          Freq         Total Category  logprob  loglift
4399            ax  53235.000000  53235.000000  Default  30.0000  30.0000
17            line  17060.000000  17060.000000  Default  29.0000  29.0000
208          write  13405.000000  13405.000000  Default  28.0000  28.0000
142   organization   9429.000000   9429.000000  Default  27.0000  27.0000
45         article   9896.000000   9896.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
4946            tv      0.020348      1.258241  Topic20 -11.0492   2.7917
6485   advertising      0.020348      1.258227  Topic20 -11.0492   2.7917
7775          logo      0.020348      1.258225  Topic20 -11.0492   2.7917
7567        circle      0.020348      1.258226  Topic20 -11.0492   2.7917
4391             _      0.020348      1.258251  Topic20 -11.0492   2.7916

[871 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
495       9  0.990580    absolute
1354      2  0.999138      accept
1277     15  0.843120  accessible
2283      6  0.995016  accomplish
183       1  0.170685     address
...     ...       ...         ...
40        8  0.002984        year
40        9  0.134080        year
614       1  0.590722       young
614       8  0.374635       young
614       9  0.031965       young

[1018 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 4, 12, 7, 14, 11, 1, 10, 2, 20, 17, 5, 3, 6, 9, 18, 16, 8, 15, 13])

    The plot on the left side shows a set of bubbles, each representing a topic.
    Larger bubbles indicate that the corresponding topic is more prevalent in the corpus of text being analyzed.
    A well-designed topic model will have large, non-overlapping bubbles spread evenly throughout the plot.
    A model with too many topics will have many small, overlapping bubbles clustered together in one part of the plot.
    When you hover over a bubble, the words and bars on the right-hand side of the plot will change. These words are the most important keywords that define the selected topic.
    The topic model created using pyLDAvis looks good and is effective in identifying natural topics in the document.
    To improve the model, the Mallet version of LDA algorithm will be used, and the focus will be on identifying the optimal number of topics for a given corpus of text.

### Building LDA Mallet Model

In [74]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip

mallet_path ="C:/Users/soulo/MACHINE_LEARNING/Topic Modeling/data/mallet-2.0.8/bin/mallet"
# mallet_path ="C:/Users/soulo/MACHINE_LEARNING/Topic Modeling/data/mallet-2.0.8/bin"

# mallet_path ="C:/Users/soulo/MACHINE_LEARNING/Topic Modeling/data/mallet-2.0.8"
# mallet_path ="C:/Users/soulo/MACHINE_LEARNING/Topic Modeling/data"


# ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [70]:
# !pip install --upgrade gensim 

# !pip install gensim

In [76]:
from gensim.models.wrappers import LdaMallet
ldamallet = gensim.models.wrappers.ldamallet.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)


ModuleNotFoundError: No module named 'gensim.models.wrappers'

In [77]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

NameError: name 'ldamallet' is not defined

In [ ]:
###

In [78]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = []
    model_list = []
    
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
###

In [ ]:
###

In [ ]:
###

In [ ]:
###

In [ ]:
###

In [ ]:
###

In [ ]:
###

In [ ]:
###

In [ ]:
###

In [ ]:
###

In [ ]:
###

In [ ]:
###

In [ ]:
###